In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection
from sklearn import linear_model
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline


In [13]:

data1 = pd.read_csv('reprocessed.hungarian.data',sep=' ')
data1 = np.array(data1.dropna())
print(data1)
X = np.array(data1[:,:11])
Y = np.array(data1[:,12])

[[49.  0.  3. ... -9. -9.  1.]
 [37.  1.  2. ... -9. -9.  0.]
 [48.  0.  4. ... -9. -9.  3.]
 ...
 [48.  1.  3. ... -9.  6.  0.]
 [47.  0.  2. ... -9. -9.  0.]
 [53.  1.  4. ... -9. -9.  0.]]


In [14]:
eBAG = []
eFORC = []
eOOBBag = []
eOOBFRC = []

n_boot = 300
for i in range(n_boot):
    bstrRegr = BaggingClassifier(DecisionTreeRegressor(),n_estimators=n_boot)
    bOOBRegr = BaggingClassifier(DecisionTreeRegressor(),n_estimators=n_boot,oob_score=True)
    X_train, X_validation, Y_train, Y_validation = sklearn.model_selection.train_test_split(X, Y, test_size=0.5)
    bstrRegr.fit(X_train, Y_train)
    camaleon = bstrRegr.predict_proba(X_validation)
    eBAG.append(log_loss(Y_validation,camaleon))
    bOOBRegr.fit(X_train, Y_train)
    camaleon1 = bOOBRegr.predict_proba(X_validation)
    eOOBBag.append(log_loss(Y_validation,camaleon1))
    clf = RandomForestClassifier(n_estimators=int(np.sqrt(n_boot)))
    clfoob = RandomForestClassifier(n_estimators=int(np.sqrt(n_boot)),oob_score=True)
    clf.fit(X_train, Y_train) 
    camaleon2 = clf.predict_proba(X_validation)
    eFORC.append(log_loss(Y_validation,camaleon2))
    clfoob.fit(X_train, Y_train) 
    camaleon3 = clfoob.predict_proba(X_validation)
    eOOBFRC.append(log_loss(Y_validation,camaleon3))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
x=np.arange(n_boot)
plt.plot(x, eBAG, c = 'black')
plt.plot(x,eOOBBag, c='g')
plt.plot(x, eFORC,  c='orange')
plt.plot(x,eOOBBag, c='blue')